In [1]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import SE3, SO3
from scipy.interpolate import CubicSpline
from roboticstoolbox.backends.swift import Swift
import matplotlib.pyplot as plt
import time
from spatialgeometry import Cuboid

Robot in the envirnoment

In [2]:
# Initialize Swift 3D visualization environment
env = Swift()

# Launch browser window at http://localhost:9000
env.launch(realtime=True,comms="rtc",browser="http://localhost:9000")
#env.launch()

# Create Panda robot model
panda = rtb.models.Panda()

# Define home configuration (ready pose, 7 joint angles in radians)
q_home = np.array([0, -np.pi/4, 0, -3*np.pi/4, 0, np.pi/2, np.pi/4])

# Add robot to Swift environment for 3D visualization
env.add(panda)


Could not open specified browser, using default instead



0

Objects in the environment

In [3]:

# Dictionary storing object names and their (pick_position, place_position) coordinates
objects = {
    'red':    (np.array([0.5, 0.2, 0.05]), np.array([0.3, -0.3, 0.05])),
    'blue':   (np.array([0.6, 0.0, 0.05]), np.array([0.3, -0.5, 0.05])),
    'green':  (np.array([0.4, 0.3, 0.05]), np.array([0.1, -0.3, 0.05])),
    'yellow': (np.array([0.55, 0.1, 0.05]), np.array([0.1, -0.5, 0.05]))
}

# Print total number of objects to sort
print(f"Objects to sort: {len(objects)}")

# Loop through each object and print its pick and place positions
for name, (pick, place) in objects.items():
    print(f"  {name}: pick={pick}, place={place}")

# Create visible cube objects at pick positions with color
bin_red = Cuboid([0.1, 0.1, 0.1], pose=SE3(0.5, 0.2, 0.05), color=[1, 0, 0, 0.8])      
bin_blue = Cuboid([0.1, 0.1, 0.1], pose=SE3(0.6, 0.0, 0.05), color=[0, 0, 1, 0.8])     
bin_green = Cuboid([0.1, 0.1, 0.1], pose=SE3(0.4, 0.3, 0.05), color=[0, 1, 0, 0.8])    
bin_yellow = Cuboid([0.1, 0.1, 0.1], pose=SE3(0.55, 0.1, 0.05), color=[1, 1, 0, 0.8])  

# Add each cube object to Swift 3D environment
env.add(bin_red)
env.add(bin_blue)
env.add(bin_green)
env.add(bin_yellow)

# Confirm all objects were successfully added
print("[OK] Objects added to Swift")

Objects to sort: 4
  red: pick=[0.5  0.2  0.05], place=[ 0.3  -0.3   0.05]
  blue: pick=[0.6  0.   0.05], place=[ 0.3  -0.5   0.05]
  green: pick=[0.4  0.3  0.05], place=[ 0.1  -0.3   0.05]
  yellow: pick=[0.55 0.1  0.05], place=[ 0.1  -0.5   0.05]
[OK] Objects added to Swift


# Compute the trajectory that the robot need to do for picking 

## Functions

non molto bene

In [4]:
def compute_ik(panda, position, z_offset=0.1):
    """Compute IK for a position slightly above the object"""
    target = np.array(position).copy()
    target[2] += z_offset      
    pose = SE3.Rx(np.pi) @ SE3(target)
    result = panda.ikine_LM(pose, q0=q_home, joint_limits=True)
    return result.q, result.success

def make_traj(q_start, q_goal, duration=2.0, n_points=100):
    """Generate a smooth joint-space trajectory from q_start to q_goal."""
    t = np.linspace(0, duration, n_points)          # time vector
    traj = np.zeros((n_points, 7))                  # 7 joints
    
    for i in range(7):
        # cubic spline between start and goal for joint i
        cs = CubicSpline([0, duration], [q_start[i], q_goal[i]])
        traj[:, i] = cs(t)
    
    return traj

## Dictionary with trajectory

In [5]:
trajectories = {}

for name, (pick_pos, place_pos) in objects.items():
    # 1) IK for pick and place
    q_pick, _ = compute_ik(panda, pick_pos)
    q_place, _ = compute_ik(panda, place_pos)

    # 2) Joint-space trajectories
    traj_home_pick  = make_traj(q_home,  q_pick)   # home -> pick
    traj_pick_place = make_traj(q_pick,  q_place)  # pick -> place
    traj_place_home = make_traj(q_place, q_home)   # place -> home

    # 3) Save everything in a dictionary
    trajectories[name] = {
        "home_pick":   traj_home_pick,
        "pick_place":  traj_pick_place,
        "place_home":  traj_place_home,
    }


## Do the movment

In [6]:
import time

for name in trajectories.keys():
    for phase in ["home_pick", "pick_place", "place_home"]:
        for q in trajectories[name][phase]:
            panda.q = q
            env.step(0.01)   # aggiorna Swift
            time.sleep(0.02) # pausa 20 ms per rallentare

KeyboardInterrupt: 